In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
import hyperopt
import optuna

%matplotlib inline
plt.style.use('seaborn')

In [5]:
molecules = pd.read_csv("data/_train_sem09 (1).csv")

In [6]:
molecules.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [7]:
molecules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3751 entries, 0 to 3750
Columns: 1777 entries, Activity to D1776
dtypes: float64(942), int64(835)
memory usage: 50.9 MB


Так как предварительная обработка не требуется, данные уже закодированы и нормализованы, приступим к обучению моделей.

In [8]:
X = molecules.drop("Activity", axis=1)
y = molecules["Activity"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Обучим модели без оптимизации, получим метрики для дальнейшего сравнения.

# Логистическая регрессия без оптимизации внешних параметров

In [54]:
log_reg = linear_model.LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [55]:
print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, log_reg.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, log_reg.predict(X_test))))

F1-score на обучающем наборе: 0.892

F1-score на обучающем наборе: 0.792


# Случайный лес без оптимизации внешних параметров

In [56]:
forest = ensemble.RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [57]:
print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, forest.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, forest.predict(X_test))))

F1-score на обучающем наборе: 1.000

F1-score на обучающем наборе: 0.809


Думаю, столь высокие показатели метрик получаются из-за "стерильности" данных.

# Логистическая регрессия по сетке с кросс-валидацией

In [ ]:
hyperparams = {
    "penalty":["l2", "none"],
    "solver":["lbfgs", "sag"]
}

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000),
    param_grid = hyperparams,
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

In [59]:
print("Наилучшие значения параметров:\n{}".format(grid_search.best_params_))

Наилучшие значения параметров:
{'penalty': 'l2', 'solver': 'sag'}


In [60]:
print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, grid_search.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, grid_search.predict(X_test))))

F1-score на обучающем наборе: 0.891

F1-score на обучающем наборе: 0.793


Метрика на тестовом наборе чуть-чуть выросла, а на обучающем наборе чуть-чуть упала))

# Случайный лес по сетке с кросс-валидацией

In [70]:
hyperparams = {"n_estimators": [100, 200, 300, 400],
               "criterion":["gini", "entropy"],
               "max_depth" : [10, 15, 20, 25],
              "min_samples_leaf" : [5,7]
              }

grid_search = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_grid = hyperparams,
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 15, 20, 25],
                         'min_samples_leaf': [5, 7],
                         'n_estimators': [100, 200, 300, 400]})

In [71]:
print("Наилучшие значения параметров:\n{}".format(grid_search.best_params_))

Наилучшие значения параметров:
{'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 400}


In [72]:
print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, grid_search.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, grid_search.predict(X_test))))

F1-score на обучающем наборе: 0.942

F1-score на обучающем наборе: 0.830


Метрика на тестовом наборе заметно выросла, и от переобучения избавились.

# Логистическая регрессия по рандомизированному поиску

In [ ]:
params = {
    "penalty":["l2", "none"],
    "solver":["lbfgs", "sag"],
    "C":list(np.linspace(0.01, 1, 10, dtype=float))
}

random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000),
    param_distributions=params, 
    cv=5,
    n_iter=20,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

In [74]:
print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, random_search.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, random_search.predict(X_test))))

F1-score на обучающем наборе: 0.856

F1-score на обучающем наборе: 0.800


F1-score на тестовом наборе выросла по сравнению с сеткой, а на обучающем наборе упала - снизили вероятность переобучения.

# Случайный лес по рандомизированному поиску

In [78]:
hyperparams = {"n_estimators": list(np.linspace(100, 400, 4, dtype=int)),
               "criterion":["gini", "entropy"],
              "max_depth": list(np.linspace(1, 30, 50, dtype=int)),
              "min_samples_leaf" : list(np.linspace(1, 7, 7, dtype=int))
              }

random_search = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_distributions=hyperparams, 
    cv=5,
    n_iter=20,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 1, 2, 2, 3, 3, 4, 5, 5,
                                                      6, 6, 7, 8, 8, 9, 9, 10,
                                                      11, 11, 12, 12, 13, 14,
                                                      14, 15, 15, 16, 16, 17,
                                                      18, ...],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7],
                                        'n_estimators': [100, 200, 300, 400]})

In [79]:
print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, random_search.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, random_search.predict(X_test))))

F1-score на обучающем наборе: 0.992

F1-score на обучающем наборе: 0.824


На тестовой выборке метрика упала. Лучше использовать алгоритм по сетке с кросс-валидацией.

# Логистическая регрессия по Hyperopt

In [2]:
from hyperopt import hp, fmin, tpe, Trials

In [55]:
space = { "penalty": hp.choice(label='penalty', options=["l2", "none"]),
         "solver": hp.choice(label="solver", options=["lbfgs", "sag"]),
         "C": hp.uniform('C', 0.01, 1)
         }

random_state = 42

def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {
        "penalty": str(params["penalty"]),
        "solver": str(params["solver"]),
        "C": float(params["C"])
        }
    
    model = linear_model.LogisticRegression(**params, random_state=random_state)
    model.fit(X, y)
    
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    
    return -score

In [ ]:
trials = Trials()

best = fmin(
    hyperopt_rf,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

In [61]:
print("Наилучшие значения гиперпараметров:{}".format(best))

Наилучшие значения гиперпараметров:{'C': 0.0900747757567953, 'penalty': 0, 'solver': 0}


In [65]:
model = linear_model.LogisticRegression(
    random_state=random_state,
    penalty="l2",
    solver="lbfgs",
    C=float(best["C"]),
    max_iter=1000
)

model.fit(X_train, y_train)

print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, model.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, model.predict(X_test))))

F1-score на обучающем наборе: 0.849

F1-score на обучающем наборе: 0.799


Результат метрики логистической регрессии по hyperopt почти такой же, как логистической регрессии по рандомизированному поиску

# Случайный лес по Hyperopt

In [79]:
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

random_state = 42

def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    model.fit(X, y)

    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [82]:
trials = Trials()

best = fmin(
    hyperopt_rf,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

100%|██████████| 50/50 [04:29<00:00,  5.39s/trial, best loss: -0.815259903797766] 


In [84]:
print("Наилучшие значения гиперпараметров:{}".format(best))

Наилучшие значения гиперпараметров:{'max_depth': 20.0, 'min_samples_leaf': 3.0, 'n_estimators': 175.0}


In [88]:
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

model.fit(X_train, y_train)

print("F1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_train, model.predict(X_train))))
print("\nF1-score на обучающем наборе: {:.3f}".format(metrics.f1_score(y_test, model.predict(X_test))))

F1-score на обучающем наборе: 0.975

F1-score на обучающем наборе: 0.825


Метрика модели RandomForestClassifier на тестовой выборке почти такая же (на 0.001 выше), как у модели случайного леса по рандомизированному поиску.   
Но вот по обучающему набору - заметно ниже, значит, модель по Hyperopt менее подверженна переобучению.

# Логистическая регрессия по Optuna

In [10]:
def optuna_rf(trial, cv=5, X=X_train, y=y_train, random_state=42):
  penalty = trial.suggest_categorical('penalty', ["l2", "none"])
  solver = trial.suggest_categorical('solver', ["lbfgs", "sag"])
  C = trial.suggest_float('C', 0.01, 1, log=False)

  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state,
                                          max_iter=1000)
  model.fit(X_train, y_train)
  
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score
  

In [11]:
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
study.optimize(optuna_rf, n_trials=50)

[I 2022-09-06 20:14:30,447] A new study created in memory with name: LogisticRegression
[I 2022-09-06 20:15:28,982] Trial 0 finished with value: 0.7691121984763665 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.5848180071596476}. Best is trial 0 with value: 0.7691121984763665.
/home/roman/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/roman/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/roman/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/roman/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio

In [12]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.3f}".format(study.best_value))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'sag', 'C': 0.04356151802753293}
f1_score на обучающем наборе: 0.783


In [14]:
model = linear_model.LogisticRegression(**study.best_params,random_state=42, max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(C=0.04356151802753293, max_iter=1000, random_state=42,
                   solver='sag')

In [15]:
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, model.predict(X_test))))

f1_score на тестовом наборе: 0.797


Модель логистической регрессии по Optuna имеет метрику на тестовом наборе, практически похожую на Hyperopt (немного меньше).

# Случайный лес с Optuna

In [16]:
def optuna_rf(trial, cv=5, X=X_train, y=y_train, random_state=42):
    
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  
  model.fit(X, y)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()


  return score

In [17]:
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
study.optimize(optuna_rf, n_trials=50)

[I 2022-09-06 21:01:04,135] A new study created in memory with name: RandomForestClassifier
[I 2022-09-06 21:01:10,666] Trial 0 finished with value: 0.8002687713316747 and parameters: {'n_estimators': 146, 'max_depth': 13, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8002687713316747.
[I 2022-09-06 21:01:15,734] Trial 1 finished with value: 0.7936501629074433 and parameters: {'n_estimators': 178, 'max_depth': 14, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.8002687713316747.
[I 2022-09-06 21:01:19,124] Trial 2 finished with value: 0.7980663022949532 and parameters: {'n_estimators': 109, 'max_depth': 26, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.8002687713316747.
[I 2022-09-06 21:01:25,388] Trial 3 finished with value: 0.814831131499399 and parameters: {'n_estimators': 179, 'max_depth': 23, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.814831131499399.
[I 2022-09-06 21:01:28,834] Trial 4 finished with value: 0.8006409714368706 and parameters: {'n_es

In [18]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.3f}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 179, 'max_depth': 23, 'min_samples_leaf': 3}
f1_score на обучающем наборе: 0.815


In [19]:
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, model.predict(X_test))))

f1_score на тестовом наборе: 0.797


Метрика на тестовом наборе 0.797, что ниже, чем у модели с Hyperopt. Возможно, это связано с удалением «плохих» точек пространства из рассмотрения, либо некорректно построено пространство поиска.